In [ ]:
import numpy as np
import xarray as xr

fcf = np.random.randn(10, 10)
deltaVV = np.random.randn(10, 10, 6)
deltaCR = np.random.randn(10, 10, 6)

times = [np.datetime64(t) for t in ['2020-01-01','2020-01-02', '2020-01-07','2020-01-08', '2020-01-14', '2020-01-15']]
x = np.linspace(0, 9, 10)
y = np.linspace(10, 19, 10)
lon, lat = np.meshgrid(x, y)

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.processing.wet_snow import id_new_wet_snow
from spicy_snow.processing.snow_index import calc_delta_gamma, clip_delta_gamma_outlier, calc_snow_index

ds = xr.Dataset(data_vars = dict(
                fcf = (["x", "y"], fcf),
                deltaVV = (["x", "y", "time"], deltaVV),
                deltaCR = (["x", "y", "time"], deltaCR),
            ),
       coords = dict(
                lon = (["x", "y"], lon),
                lat = (["x", "y"], lat),
                band = ['VV', 'VH', 'inc'],
                time = times,
                relative_orbit = (["time"], [24,1,24,1,24,1])) 
)

ds = calc_delta_gamma(ds)
ds = clip_delta_gamma_outlier(ds)
ds = calc_snow_index(ds)

In [ ]:
ds